In [ ]:
import requests
import gradio as gr
import time

def request_document_intelligence(image_path):

    # 엔드포인트 정보 추가
    endpoint = "https://7ai015-document-intelligence.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read:analyze?_overload=analyzeDocument&api-version=2024-11-30"

    # 헤더 정보를 추가해야 한다.
    headers = {
        "Ocp-Apim-Subscription-Key": "6GpHBlkB9FNrHOBbpQUv8bZcYvTJHbrzmGNSy0ADpWyrg8xI7bPRJQQJ99BFACYeBjFXJ3w3AAALACOG9wyJ",
        "Content-Type": "image/png" 
    }
    
    # 바디에 대한 정보를 추가
    # body = {
    #     # "urlSource": "https://i.ytimg.com/vi/xYtjQEpQX1w/hqdefault.jpg"
    #     "urlSource": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/read.png"
    # }

    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # requests 라이브러리를 이용해서 post로 요청을 보낸다.
    response = requests.post(endpoint, headers=headers, data=image_data)

    if response.status_code !=202:
        print(f'Error: {response.status_code}')
        print("Error: {}".format(response.status_code))
        return None
    
    # 서버로부터 받은 데이터, 즉 헤더에 있는 operation-location 값을 추출한다.
    url = response.headers['Operation-Location']
    print(url)

    while True:
        # 추출된 operation-location 값을 이용해서 
        # GET 요청을 보낸다.
        result_response = requests.get(url, headers=headers)

        if result_response.status_code != 200:
            print(f"Error: {result_response.status_code}")
            return None
        
        result_response_json = result_response.json()
        current_status = result_response_json.get("status")

        # 요청을 보낸 후 받은 응답의 상태(status) 값이 running인지 확인한다.
        # running이라면, 다시 GET 요청을 보낸다.
        if current_status == "running":
            time.sleep(1) # 원래라면 이렇게 하는 게 맞다.
            print("Current status: {}".format(current_status))
            continue
        else:
            break

    # 요청을 보낸 후 받은 응답의 상태(status) 값이 succeeded인지 확인한다.
    # succeeded라면, 응답의 결과를 출력한다.      
    if current_status == 'succeeded':
        # print("Result: {}".format(current_status))
        return result_response_json
    else:
        # print("Result: {}".format(current_status))
        return None    


def random_color():
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))


def get_font():
    from PIL import ImageFont
    import platform
    
    font_size = 20
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else: # Linux
            return ImageFont.truetype("", font_size)
    except IOError:
        return ImageFont.load_default(size=font_size)


def draw_polygon(image_path, response_data):
    from PIL import Image, ImageDraw

    # PIL 라이브러리 객체를 만들어 준다.
    image = Image.open(image_path)

    # 이미지를 그릴 수 있는 객체를 만들어 준다.
    draw = ImageDraw.Draw(image)

    # response_data에서 polygone 정보를 추출해서,
    block_list = response_data['analyzeResult']['paragraphs']

    for block in block_list:
        
        line_color = random_color()
        font = get_font()

        # polygone 정보 추출
        polygon = block['boundingRegions'][0]['polygon']
        # content 정보 추출
        content = block['content']
        # (x, y) 튜플 리스트로 변환
        polygon_points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]

        draw.polygon(polygon_points, outline="red", width=5)
        # print(polygon, content)
        draw.text((polygon[0], polygon[1] - 20), content, fill=line_color, font=font)

    return image


with gr.Blocks() as demo:

    def change_image(image_path):
        # image_path를 request_document_intelligence에 전달
        # response_data = request_document_intelligence로부터 전달 받음
        response_data = request_document_intelligence(image_path)

        # response_data를 이용해서 draw_polygon을 호출
        image = draw_polygon(image_path, response_data)

        # 호출하는데 draw_polygon에 image_path를 같이 전달합니다. with response_data
        # draw_polygon은 image_path를 받아서 polygon을 그린 이미지를 반환합니다.
        # 단, draw_polygon의 리턴 형태는 pil 이미지다.
        # pil 이미지를 ouput_image가 전달 받을 수 있도록, 리턴해주면 완료.
        return image
    
    with gr.Row():
        input_image = gr.Image(label="이미지 선택", type="filepath", width=600)
        output_image = gr.Image(label="결과 이미지", type="pil", interactive=False, width=600)

    input_image.change(change_image, inputs=[input_image], outputs=[output_image])

demo.launch()

# change_image("C:/Users/USER/Downloads/카드뉴스_한글날_1_1.jpg")

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


https://7ai015-document-intelligence.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read/analyzeResults/7de51936-9c58-4df8-92c4-cad59091a562?api-version=2024-11-30
